This notebook aims to calculate the uncertainty of LNO2 PE caused by:

- peak width of a priori LNO2 profile (160 hPa, 180 hPa, and 200 hPa)
- SCD background (10th and 30th percentile)
- NO2 lifetime (2 hour, 3 hour, and 6 hour)
- IC:CG (2:1, 1:1, and 1:2)
- confidence level for LNO$_2$ selection (80% and 70%)
- cloud pressure ($\pm$ 50 hPa)

In [1]:
import pandas as pd
import numpy as np

In [2]:
from shapely.prepared import prep
import cartopy.io.shapereader as shpreader
from shapely.ops import unary_union
import shapely.vectorized

def get_geom(name, category='physical', resolution='50m'):
    # https://stackoverflow.com/questions/47894513/
    #    checking-if-a-geocoordinate-point-is-land-or-ocean-with-cartopy
    shp_fname = shpreader.natural_earth(name=name,
                                        resolution=resolution,
                                        category=category
                                       )
    if name == 'coastline':
        # https://shapely.readthedocs.io/en/stable/manual.html#object.buffer
        geom = unary_union(list(shpreader.Reader(shp_fname).geometries())).buffer(0.5)
    else:
        geom = unary_union(list(shpreader.Reader(shp_fname).geometries()))

    return prep(geom)

land_geom = get_geom('land')
coast_geom = get_geom('coastline')
ocean_geom = get_geom('ocean')

In [3]:
def filter_case(df):
    df_lno2 = df[(df['pe_lno2']>0)&(df['pe_lno2geo']>0)&(df['pe_lno2vis'])>0]

    return df_lno2


def read_data(filename):
    df = pd.read_csv(filename)
    df = filter_case(df)

    cond_coast = shapely.vectorized.contains(coast_geom, df['longitude'], df['latitude'])
    cond_land = shapely.vectorized.contains(land_geom, df['longitude'], df['latitude'])
    cond_ocean = shapely.vectorized.contains(ocean_geom, df['longitude'], df['latitude'])

    region = np.argmax(np.vstack((cond_coast, cond_land, cond_ocean))==1, axis=0)
    df['region'] = region

    # df['time'] = pd.to_datetime(df['time'])
    # df['lfr'] = df['nlightning']/df_lno2['area']

    return df


def correct_lightning(df, coeff=4):
    # corrected stroke considering the detection efficiency
    # IC:CG 1) 1:1 -> 4; 2) 2:1 -> 6 3) 1:2 -> 3
    df['nlightning'] *= coeff
    df['pe_lno2'] /= coeff


def perturb(df):
    # 0: 'coast', 1: 'land', 2: 'ocean'
    pe_coast = df['pe_lno2'][df['region']==0].median()
    pe_land = df['pe_lno2'][df['region']==1].median()
    pe_ocean = df['pe_lno2'][df['region']==2].median()
    pe = df['pe_lno2'].median()

    return pe_coast, pe_land, pe_ocean, pe

## Default

In [4]:
df_lno2 = read_data('../data/S5P_LNO2_production.csv')

correct_lightning(df_lno2, coeff=4)

pe_coast, pe_land, pe_ocean, pe = perturb(df_lno2)

## Uncertainty

In [5]:
def reldiff(v1, v2, v3):
    return (((v1-v3)/v3-(v2-v3)/v3)/2)

def reldiff2(v1, v2):
    return (v1-v2)/v2

### peak width

In [6]:
df_lno2_high_pw = read_data('../data/S5P_LNO2_production_peakwidth200.csv')
correct_lightning(df_lno2_high_pw, coeff=4)

df_lno2_low_pw = read_data('../data/S5P_LNO2_production_peakwidth160.csv')
correct_lightning(df_lno2_low_pw, coeff=4)

In [7]:
pe_coast_high_pw, pe_land_high_pw, pe_ocean_high_pw, pe_high_pw = perturb(df_lno2_high_pw)
pe_coast_low_pw, pe_land_low_pw, pe_ocean_low_pw, pe_low_pw = perturb(df_lno2_low_pw)

In [8]:
print('Uncertainty (%)',
'Coast: ' ,reldiff(pe_coast_high_pw, pe_coast_low_pw, pe_coast)*100,
'Land: ' ,reldiff(pe_land_high_pw, pe_land_low_pw, pe_land)*100,
'Ocean: ' ,reldiff(pe_ocean_high_pw, pe_ocean_low_pw, pe_ocean)*100,
'All: ' ,reldiff(pe_high_pw, pe_low_pw, pe)*100,
)

uncert_pw = reldiff(pe_high_pw, pe_low_pw, pe)*100

Uncertainty (%) Coast:  7.225549548122315 Land:  5.985004357092456 Ocean:  13.354405947040304 All:  7.249209819227446


### lifetime

In [9]:
df_lno2_low_tau = read_data('../data/S5P_LNO2_production_tau2.csv')
correct_lightning(df_lno2_low_tau, coeff=4)

df_lno2_high_tau = read_data('../data/S5P_LNO2_production_tau4.csv')
correct_lightning(df_lno2_high_tau, coeff=4)

pe_coast_high_tau, pe_land_high_tau, pe_ocean_high_tau, pe_high_tau = perturb(df_lno2_high_tau)
pe_coast_low_tau, pe_land_low_tau, pe_ocean_low_tau, pe_low_tau = perturb(df_lno2_low_tau)

In [10]:
print('Uncertainty (%)',
'Coast: ' ,reldiff(pe_coast_low_tau, pe_coast_high_tau, pe_coast)*100,
'Land: ' ,reldiff(pe_land_low_tau, pe_land_high_tau,  pe_land)*100,
'Ocean: ' ,reldiff(pe_ocean_low_tau, pe_ocean_high_tau,  pe_ocean)*100,
'All: ' ,reldiff(pe_low_tau, pe_high_tau, pe)*100,
)

uncert_lifetime = reldiff(pe_low_tau, pe_high_tau, pe)*100

Uncertainty (%) Coast:  15.8349180220144 Land:  155.2413471611436 Ocean:  36.99485890518229 All:  36.47939717311078


### background

In [11]:
df_lno2_low_bkgd = read_data('../data/S5P_LNO2_production_bkgd10.csv')
correct_lightning(df_lno2_low_bkgd, coeff=4)

pe_coast_low_bkgd, pe_land_low_bkgd, pe_ocean_low_bkgd, pe_low_bkgd = perturb(df_lno2_low_bkgd)

In [12]:
print('Uncertainty (%)',
'Coast: ' ,reldiff2(pe_coast_low_bkgd, pe_coast)*100,
'Land: ' ,reldiff2(pe_land_low_bkgd, pe_land)*100,
'Ocean: ' ,reldiff2(pe_ocean_low_bkgd,  pe_ocean)*100,
'All: ' ,reldiff2(pe_low_bkgd, pe)*100,
)

uncert_bkgd = reldiff2(pe_low_bkgd, pe)*100

Uncertainty (%) Coast:  23.39541119415398 Land:  58.58515649056712 Ocean:  12.819917513210791 All:  1.769017507145169


### IC:CG

In [13]:
df_lno2_low_ratio = read_data('../data/S5P_LNO2_production.csv')
correct_lightning(df_lno2_low_ratio, coeff=3)
pe_coast_low_ratio, pe_land_low_ratio, pe_ocean_low_ratio, pe_low_ratio = perturb(df_lno2_low_ratio)

df_lno2_high_ratio = read_data('../data/S5P_LNO2_production.csv')
correct_lightning(df_lno2_high_ratio, coeff=6)
pe_coast_high_ratio, pe_land_high_ratio, pe_ocean_high_ratio, pe_high_ratio = perturb(df_lno2_high_ratio)

In [14]:
print('Uncertainty (%)',
'Coast: ' ,reldiff(pe_coast_low_ratio, pe_coast_high_ratio, pe_coast)*100,
'Land: ' ,reldiff(pe_land_low_ratio, pe_land_high_ratio,  pe_land)*100,
'Ocean: ' ,reldiff(pe_ocean_low_ratio, pe_ocean_high_ratio,  pe_ocean)*100,
'All: ' ,reldiff(pe_low_ratio, pe_high_ratio, pe)*100,
)

uncert_ratio = reldiff(pe_low_ratio, pe_high_ratio, pe)*100

Uncertainty (%) Coast:  33.33333333333333 Land:  33.33333333333333 Ocean:  33.333333333333336 All:  33.33333333333333


### confidence level of selection

In [15]:
df_lno2_high_alpha = read_data('../data/S5P_LNO2_production_alpha30.csv')
correct_lightning(df_lno2_high_alpha, coeff=4)

pe_coast_high_alpha, pe_land_high_alpha, pe_ocean_high_alpha, pe_high_alpha = perturb(df_lno2_high_alpha)

In [16]:
print('Uncertainty (%)',
'Coast: ' ,reldiff2(pe_coast_high_alpha, pe_coast)*100,
'Land: ' ,reldiff2(pe_land_high_alpha, pe_land)*100,
'Ocean: ' ,reldiff2(pe_ocean_high_alpha,  pe_ocean)*100,
'All: ' ,reldiff2(pe_high_alpha, pe)*100,
)

uncert_alpha = reldiff2(pe_high_alpha, pe)*100

Uncertainty (%) Coast:  1.6163705919000984 Land:  223.72776191159588 Ocean:  26.70601416007437 All:  -12.681077016229139


### Cloud pressure

In [17]:
df_lno2_low_cp = read_data('../data/S5P_LNO2_production_peakoffest50.csv')
correct_lightning(df_lno2_low_cp, coeff=4)

df_lno2_high_cp = read_data('../data/S5P_LNO2_production_peakoffest-50.csv')
correct_lightning(df_lno2_high_cp, coeff=4)

pe_coast_high_cp, pe_land_high_cp, pe_ocean_high_cp, pe_high_cp = perturb(df_lno2_high_cp)
pe_coast_low_cp, pe_land_low_cp, pe_ocean_low_cp, pe_low_cp = perturb(df_lno2_low_cp)

In [18]:
print('Uncertainty (%)',
'Coast: ' ,reldiff(pe_coast_low_cp, pe_coast_high_cp, pe_coast)*100,
'Land: ' ,reldiff(pe_land_low_cp, pe_land_high_cp,  pe_land)*100,
'Ocean: ' ,reldiff(pe_ocean_low_cp, pe_ocean_high_cp,  pe_ocean)*100,
'All: ' ,reldiff(pe_low_cp, pe_high_cp, pe)*100,
)

uncert_cp = reldiff(pe_low_cp, pe_high_cp, pe)*100

Uncertainty (%) Coast:  12.242449786730651 Land:  13.728866591924499 Ocean:  -11.217293390862537 All:  7.46665239185912


### Total uncetainty

In [19]:
uncert_other = 10
np.sqrt(uncert_pw**2+uncert_lifetime**2+uncert_bkgd**2+uncert_alpha**2+uncert_ratio**2+uncert_cp**2+uncert_other**2)/100

0.5304807826303262

In [20]:
print('peak width: ', uncert_pw, 'lifetime: ', uncert_lifetime,
'background: ', uncert_bkgd, 'LNO2 selection: ', uncert_alpha, 'IC:CG: ', uncert_ratio,
'Cloud pressure: ', uncert_cp, 'Others: ', uncert_other)

peak width:  7.249209819227446 lifetime:  36.47939717311078 background:  1.769017507145169 LNO2 selection:  -12.681077016229139 IC:CG:  33.33333333333333 Cloud pressure:  7.46665239185912 Others:  10
